In [75]:
import numpy as np
import pandas as pd 
import requests
import xlsxwriter
import math
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame 
%matplotlib inline

In [76]:
load_dotenv()
api_key = os.getenv('ALPACA_API_KEY')
api_secret = os.getenv('ALPACA_SECRET_KEY')

In [81]:
base_url = 'https://paper-api.alpaca.markets'

api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

account = api.get_account()





# Create the Alpaca API object
# YOUR CODE HERE!
spy_price = api.get_bars('SPY',  TimeFrame.Day, "2010-06-08", "2022-06-08")
spy_df = spy_price.df

print(spy_df)

                             open      high       low   close     volume  \
timestamp                                                                  
2015-12-01 05:00:00+00:00  209.42  210.8200  209.1100  210.68   97858418   
2015-12-02 05:00:00+00:00  210.60  211.0000  208.2300  208.54  108069059   
2015-12-03 05:00:00+00:00  208.90  209.1500  204.7511  205.58  166224154   
2015-12-04 05:00:00+00:00  206.10  209.9700  205.9300  209.66  192878747   
2015-12-07 05:00:00+00:00  209.20  209.7295  207.2000  208.27  102027111   
...                           ...       ...       ...     ...        ...   
2022-06-02 04:00:00+00:00  409.58  417.4400  407.0400  417.39   78866434   
2022-06-03 04:00:00+00:00  412.47  414.0400  409.5100  410.54   68321202   
2022-06-06 04:00:00+00:00  414.76  416.6090  410.5523  411.79   56893593   
2022-06-07 04:00:00+00:00  408.13  416.2200  407.6100  415.74   57784776   
2022-06-08 04:00:00+00:00  413.93  415.8200  410.3800  411.22   64087538   

           

In [82]:
spy_df = spy_df.drop(columns=["open","high","low","volume","trade_count","vwap"])


In [83]:
spy_df


,close
timestamp,
2015-12-01 05:00:00+00:00,210.68
2015-12-02 05:00:00+00:00,208.54
2015-12-03 05:00:00+00:00,205.58
2015-12-04 05:00:00+00:00,209.66
2015-12-07 05:00:00+00:00,208.27
...,...
2022-06-02 04:00:00+00:00,417.39
2022-06-03 04:00:00+00:00,410.54
2022-06-06 04:00:00+00:00,411.79


In [87]:
spy_df["Actual Returns"] = spy_df["close"].pct_change()

In [88]:
spy_df = spy_df.dropna()

In [89]:
spy_df

,close,Signal,Actual Returns
timestamp,,,
2015-12-02 05:00:00+00:00,208.54,0.0,-0.010158
2015-12-03 05:00:00+00:00,205.58,0.0,-0.014194
2015-12-04 05:00:00+00:00,209.66,0.0,0.019846
2015-12-07 05:00:00+00:00,208.27,0.0,-0.006630
2015-12-08 05:00:00+00:00,206.99,0.0,-0.006146
...,...,...,...
2022-06-02 04:00:00+00:00,417.39,0.0,0.019043
2022-06-03 04:00:00+00:00,410.54,0.0,-0.016412
2022-06-06 04:00:00+00:00,411.79,0.0,0.003045


In [90]:
short_window = 4
long_window = 100

spy_df['SMA_Fast'] = spy_df.loc[:, ["close"]].rolling(window=short_window).mean()
spy_df['SMA_Slow'] = spy_df.loc[:, ["close"]].rolling(window=long_window).mean()

C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [91]:
spy_df = spy_df.dropna()

In [92]:
spy_df

,close,Signal,Actual Returns,SMA_Fast,SMA_Slow
timestamp,,,,,
2016-04-26 04:00:00+00:00,209.00,0.0,0.001966,208.8625,199.4766
2016-04-27 04:00:00+00:00,209.38,0.0,0.001818,208.9825,199.4850
2016-04-28 04:00:00+00:00,207.42,0.0,-0.009361,208.5975,199.5034
2016-04-29 04:00:00+00:00,206.30,0.0,-0.005400,208.0250,199.4698
2016-05-02 04:00:00+00:00,208.03,0.0,0.008386,207.7825,199.4674
...,...,...,...,...,...
2022-06-02 04:00:00+00:00,417.39,0.0,0.019043,413.7925,432.9898
2022-06-03 04:00:00+00:00,410.54,0.0,-0.016412,412.6125,432.4401
2022-06-06 04:00:00+00:00,411.79,0.0,0.003045,412.3275,431.8605


In [93]:
spy_df["Signal"] = 0.0


spy_df.loc[(spy_df["Actual Returns"] >= 0), 'Signal'] = 1
spy_df.loc[(spy_df["Actual Returns"] < 0), 'Signal'] = -1

C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

In [94]:
spy_df

,close,Signal,Actual Returns,SMA_Fast,SMA_Slow
timestamp,,,,,
2016-04-26 04:00:00+00:00,209.00,1.0,0.001966,208.8625,199.4766
2016-04-27 04:00:00+00:00,209.38,1.0,0.001818,208.9825,199.4850
2016-04-28 04:00:00+00:00,207.42,-1.0,-0.009361,208.5975,199.5034
2016-04-29 04:00:00+00:00,206.30,-1.0,-0.005400,208.0250,199.4698
2016-05-02 04:00:00+00:00,208.03,1.0,0.008386,207.7825,199.4674
...,...,...,...,...,...
2022-06-02 04:00:00+00:00,417.39,1.0,0.019043,413.7925,432.9898
2022-06-03 04:00:00+00:00,410.54,-1.0,-0.016412,412.6125,432.4401
2022-06-06 04:00:00+00:00,411.79,1.0,0.003045,412.3275,431.8605
